# Using the Twitter API

Twitter has what's called an "API", which stands for "Application Programming Interface". It lets us use Twitter through code. In our case, we're using Python code.

With the API we can do basically anything we could normally do on Twitter. We can post tweets, search tweets, favorite, retweet, follow, or unfollow, block or mute, and so on. The only difference is that instead of clicking on interface elements, we write a line or two of code.

Why is this useful? It let's us leverage the speed of a computer to do things that would take us hours, days, or even weeks to do otherwise.

For instance, if we wanted to map out a Twitter bot net, we could click around until we find a bot, then manually sift through its followers to find more bots, then sift through the followers of those bots, and so on.

Or we could write 10 or so lines of code and have the computer do the same thing in a matter of seconds.

## A note on comments

A line of code that starts with `#` is called a _comment_. It just means that Python ignores that line. We can use it to write notes about what a piece of code does. Or we can use it to temporarily turn off a line of code.

---

## Tips

Try changing stuff and seeing what happens! It's the best way to learn programming. You won't be able to "break" anything so don't worry about that.

<img src="https://s-media-cache-ak0.pinimg.com/originals/a3/0a/68/a30a68dcdf20ae4730f4c864a6e2875c.jpg" width=280>

If you encounter an error, copy and paste it into Google: often you'll see that someone ran into the same issue and posted an answer on how to fix it.

<img src="https://pbs.twimg.com/media/CfEhcE3VIAE5ePS.jpg" width=280>

If you really get stuck, send me an email and I'll see what I can do.

## A note on rate-limits

Before we go any further, we have to talk about _rate-limits_. Twitter's API is rate-limited, which means we can only take a certain amount of actions in a given period of time. Twitter does this to prevent spam.

This means if we do anything too fast or too much in a span of a few minutes (like if we lookup too many Twitter users at once), Twitter will temporarily block us (for as long as 15 minutes!) and we'll get an error in our program.

This is really inconvenient, so we need a small workaround that will help us avoid this. This is what the code below does.

It's not important that you understand this code, but __*it is important that you run it first!*__

In [1]:
import requests_cache
requests_cache.install_cache('/tmp/twitter.cache', backend='sqlite', expire_after=60*60*24*7) # expire after one week

In [2]:
# set your authentication info
CONSUMER_KEY='<YOUR CONSUMER KEY>'
CONSUMER_SECRET='<YOUR CONSUMER SECRET>'
ACCESS_TOKEN='<YOUR ACCESS TOKEN>'
ACCESS_TOKEN_SECRET='<YOUR ACCESS TOKEN SECRET>'

Below we'll create the API object that lets us use the Twitter API.

We pass in our authentication info to "login" to the API.

We also specify `sleep_on_rate_limit=True` which tells the program to wait when we hit the rate limit instead of giving an error.

In [3]:
# import the `twitter` library.
# libraries give us access to code others
# have written, so we don't have to write as much ourselves.
import twitter

api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN,
                  access_token_secret=ACCESS_TOKEN_SECRET,
                  sleep_on_rate_limit=True)

# see who you're logged in as
api.VerifyCredentials()

User(ID=354161158, ScreenName=frnsys)

Now we're logged into the API! We can start using it.

For example, we may want to fetch some tweets. There are several ways we can do this:

- we can get tweets from our own timeline by using `api.GetUserTimeline()`
- we can get tweets for a specific user by using `api.GetUserTimeline(screen_name='rihanna')` (replacing `rihanna` with another username if you'd like)
- we can get tweets by searching by using `api.GetSearch('catalonia', result_type='popular')` (replacing `catalonia` with another search term -- this can be an @ mention, a hashtag, etc)
    - `result_type` can also be `recent` or `mixed`

Each of these gives us back a list of tweets.

Below we'll get tweets for a search term and store the results in a `tweets` variable.

In [ ]:
# search for tweets
tweets = api.GetSearch('catalonia', result_type='popular', count=100)

In [ ]:
# show the text of each tweet we saved
for t in tweets:
    print(t.text)

Another powerful thing we can do with the Twitter API is see who a person is following and who they're followed by:

In [ ]:
# get the user ids of someone's followers
_, _, follower_ids = api.GetFollowerIDsPaged(screen_name='rihanna')
followers = api.UsersLookup(user_id=follower_ids[:100])
print(followers)

In [ ]:
# get the user ids of someone's friends (who they follow)
_, _, friend_ids = api.GetFriendIDsPaged(screen_name='rihanna')
friends = api.UsersLookup(user_id=friend_ids[:100])
print(friends)

In [ ]:
# according to our criteria,
# a user might be a bot if their last name ends with 8 digits.
# this function will return True if a user's name ends with 8 digits.
def maybe_bot(user):
    last_8 = user.screen_name[-8:]
    try:
        int(last_8)
        return True
    except ValueError:
        return False